In this code, we organize the data collected from the VNA through the measurement scripts. 

Because the data are not in a format compatible with the Analysis UserFile, a transition script is required to convert them into the proper format.

# Goal

The files collected from the measurement scripts are saved in folders named, for example, "5p932GHz_-25p0dBm_5avgs_201points".

Each folder contains two files:

* A CSV file named "5p932GHz_-25p0dBm_5avgs_201points_001.csv"
* A PNG file named "5p932GHz_-25p0dBm_5avgs_201points_001.png"

Our plan is to extract the CSV file from each folder and rename it to, for example, "Tony_Ta_a_plane_5p932GHz_-25dBm_13mK_.csv".

After renaming, each CSV file will be moved to the target directory, for example, "Resonator_0_5p932GHz".

## Organize and Rename the Measured Files

In [1]:
import os
import re
import glob
import shutil
import numpy as np
import pandas as pd

The format of the measured files is (four columns, with header):
* Col_0 - number (0, 1, 2, 3, ...)
* Col_1 - Frequency (in Hz)
* Col_2 - S21 magn_dB (in dB)  
* Col_3 - S21 phase_rad (in rad)

The format of the analyzed filesis (three columns, without header):
* Col_0 - Frequency (in Hz)
* Col_1 - S21 magn (in dB)  
* Col_2 - S21 phase (in degree)

In [2]:
def grab_and_rename_csvs(
        parent_folder_path,
        filename_template="Tony_Ta_c_plane_{freq}_{power}_150mK_.csv",
        target_folder=None
    ):
    
    """
    Scan subfolders, process CSVs, rename them using template move the renamed files to a target folder.
    """

    # If a target folder is given, ensure it exists
    if target_folder is not None:
        os.makedirs(target_folder, exist_ok=True)

    # List all subdirectories inside parent folder
    subfolders = [f.path for f in os.scandir(parent_folder_path) if f.is_dir()]

    for subfolder_path in subfolders:

        subfolder_name = os.path.basename(subfolder_path)

        # Extract frequency from folder name
        freq_match = re.search(r'(\d+p\d+GHz)', subfolder_name)
        if not freq_match:
            print(f"Skipping {subfolder_name}: Frequency not found.")
            continue
        freq_str = freq_match.group(1)

        # Process CSV files in this subfolder
        csv_files = glob.glob(os.path.join(subfolder_path, "*.csv"))
        for csv_file in csv_files:
            filename = os.path.basename(csv_file)

            # Extract power from filename
            power_match = re.search(r'(-?\d+)p\d+dBm', filename)
            if not power_match:
                print(f"Skipping {filename}: Power not found.")
                continue
            power_str = f"{power_match.group(1)}dBm"

            # Create new filename
            new_filename = filename_template.format(freq=freq_str, power=power_str)

            # Temporary output path (save processed file in parent folder first)
            tmp_filepath = os.path.join(parent_folder_path, new_filename)

            try:
                # --- Detect header row ---
                first_row = pd.read_csv(csv_file, nrows=1, header=None)

                def is_number(x):
                    try:
                        float(x)
                        return True
                    except:
                        return False

                cols_to_check = [c for c in [1, 2, 3] if c in first_row.columns]
                if cols_to_check:
                    numeric_mask = first_row[cols_to_check].map(is_number)
                    first_row_is_header = not numeric_mask.iloc[0].all()
                else:
                    first_row_is_header = True

                skiprows = 1 if first_row_is_header else 0

                # --- Read CSV ---
                df = pd.read_csv(
                    csv_file,
                    skiprows=skiprows,
                    header=None,
                    usecols=[1, 2, 3]
                )

                # --- Phase unit detection ---
                phase_series = df[3].astype(float)
                if (phase_series.abs() <= 180).all():
                    df[3] = np.rad2deg(phase_series)  # Convert from rad to deg

                # --- Save processed CSV ---
                df.to_csv(tmp_filepath, index=False, header=False)

                # --- Move file to target folder if requested ---
                if target_folder is not None:
                    target_path = os.path.join(target_folder, new_filename)

                    # If file exists in target folder, remove to avoid error
                    if os.path.exists(target_path):
                        os.remove(target_path)

                    shutil.move(tmp_filepath, target_path)
                    print(f"✅ Saved: {new_filename}.")

            except Exception as e:
                print(f"❌ Failed to process {filename}: {e}")

In [3]:
parent_folder_path = r"C:\Users\user\Downloads\15mK_Power_Sweep_terminate_pumpline\15mK_Power_Sweep_terminate_pumpline\02_19_0937"
target_folder = r"C:\Users\user\Documents\GitHub\Measurements\Cooldown_74_Line6-QSD_CPW_w50g25_02\most_recent_data"
grab_and_rename_csvs(
    parent_folder_path=parent_folder_path,
    filename_template="QSD_CPW_w50g25_02_{freq}_{power}_15mK.csv",
    target_folder=target_folder
)

✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-10dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-12dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-15dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-18dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-20dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-22dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-25dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-28dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-30dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-32dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-35dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-38dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-40dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-42dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-45dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-48dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-50dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-52dBm_15mK.csv.
✅ Saved: QSD_CPW_w50g25_02_4p615GHz_-55dBm_15m

## Run the Data Analysis

After organizing the data to fit the Analysis UserFile format, we then apply the Analysis UserFile to perform the data analysis.

## Collect the fitting plot from Analysis UserFile

After completing the data analysis, we want to collect all the PNG files into a single folder named "fit_plots".

In [4]:
import os
import glob
import shutil

In [5]:
def collect_png(png_folder, keyword, stored_folder_name,
                file_extension="*.png", delete_source_folders=False):

    # Build search pattern: **/*keyword*.ext
    ext = file_extension.replace("*", "")  # '*.png' -> '.png'
    pattern = os.path.join(png_folder, "**", f"*{keyword}*{ext}")
    matched_files = glob.glob(pattern, recursive=True)

    # Output folder
    target_folder = os.path.join(png_folder, stored_folder_name)
    os.makedirs(target_folder, exist_ok=True)

    copied_count = 0
    for src in matched_files:
        dst = os.path.join(target_folder, os.path.basename(src))

        # Skip if file is already in the target folder
        if os.path.abspath(src) == os.path.abspath(dst):
            continue

        shutil.copy(src, dst)
        copied_count += 1

    print(f"Copied {copied_count} files to {target_folder}")

    # Optionally delete the source folders
    if delete_source_folders:
        source_dirs = set()
        png_folder_abs = os.path.abspath(png_folder)
        target_folder_abs = os.path.abspath(target_folder)

        # Collect all parent directories that contained matched files
        for src in matched_files:
            parent = os.path.dirname(os.path.abspath(src))

            # Don't delete the main folder or the target folder
            if parent in (png_folder_abs, target_folder_abs):
                continue

            # Extra safety: only delete folders inside png_folder
            if os.path.commonpath([parent, png_folder_abs]) == png_folder_abs:
                source_dirs.add(parent)

        # Delete deeper folders first (child before parent)
        for d in sorted(source_dirs, key=len, reverse=True):
            if os.path.isdir(d):
                shutil.rmtree(d)
        print(f"Deleted {len(source_dirs)} source folders")

    return matched_files

In [7]:
png_folder = r"C:\Users\user\Documents\GitHub\Measurements\Cooldown_74_Line6-QSD_CPW_w50g25_02\Compare_wiwo_termination_pumpline\Resonator_0_4p615GHz_terminatepumpline"
keyword = "DCM"
stored_folder_name = "fit_plots"
file_extension = "*.png"
delete_source_folders = True  # <-- set to False if don't want deletion

collect_png(png_folder, keyword, stored_folder_name,
            file_extension=file_extension,
            delete_source_folders=delete_source_folders)

Copied 24 files to C:\Users\user\Documents\GitHub\Measurements\Cooldown_74_Line6-QSD_CPW_w50g25_02\Compare_wiwo_termination_pumpline\Resonator_0_4p615GHz_terminatepumpline\fit_plots
Deleted 24 source folders


['C:\\Users\\user\\Documents\\GitHub\\Measurements\\Cooldown_74_Line6-QSD_CPW_w50g25_02\\Compare_wiwo_termination_pumpline\\Resonator_0_4p615GHz_terminatepumpline\\DCM_202602019_10_35_43\\DCM_QSD_CPW_w50g25_02_4p615GHz_-10dBm_15mK.png',
 'C:\\Users\\user\\Documents\\GitHub\\Measurements\\Cooldown_74_Line6-QSD_CPW_w50g25_02\\Compare_wiwo_termination_pumpline\\Resonator_0_4p615GHz_terminatepumpline\\DCM_202602019_10_35_44\\DCM_QSD_CPW_w50g25_02_4p615GHz_-12dBm_15mK.png',
 'C:\\Users\\user\\Documents\\GitHub\\Measurements\\Cooldown_74_Line6-QSD_CPW_w50g25_02\\Compare_wiwo_termination_pumpline\\Resonator_0_4p615GHz_terminatepumpline\\DCM_202602019_10_35_45\\DCM_QSD_CPW_w50g25_02_4p615GHz_-15dBm_15mK.png',
 'C:\\Users\\user\\Documents\\GitHub\\Measurements\\Cooldown_74_Line6-QSD_CPW_w50g25_02\\Compare_wiwo_termination_pumpline\\Resonator_0_4p615GHz_terminatepumpline\\DCM_202602019_10_35_45_2\\DCM_QSD_CPW_w50g25_02_4p615GHz_-18dBm_15mK.png',
 'C:\\Users\\user\\Documents\\GitHub\\Measurements